<a href="https://colab.research.google.com/github/akash-agni/DeepLearning/blob/Development/CIFAR_100_Image_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz

--2021-02-19 17:05:32--  https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169001437 (161M) [application/x-gzip]
Saving to: ‘cifar-100-python.tar.gz’

cifar-100-python.ta 100%[===================>] 161.17M  50.0MB/s    in 3.9s    

2021-02-19 17:05:36 (41.7 MB/s) - ‘cifar-100-python.tar.gz’ saved [169001437/169001437]



In [2]:
!mkdir cifar-100
!tar -xf cifar-100-python.tar.gz -C ./cifar-100
!rm -rf cifar-100-python.tar.gz

In [40]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
from pickle import load
import matplotlib.pyplot as plt

In [7]:
workdir = os.curdir
datadir = os.path.join(workdir, 'cifar-100', 'cifar-100-python')
train_data_file = os.path.join(datadir, 'train')
test_data_file = os.path.join(datadir, 'test')
meta_data_file = os.path.join(datadir, 'meta')

In [108]:
def extract_data(filename):
  with open(filename, 'rb') as fo:
    dataset = load(fo, encoding='bytes')
  dataset = {'images':dataset[b'data'],
             'fine_labels':dataset[b'fine_labels'],
             'coarse_labels':dataset[b'coarse_labels']}
  dataset = tf.data.Dataset.from_tensors(dataset)
  return dataset

def transform_data(dataset):
  #dataset = dataset.map(lambda x: tf.reshape(x['image'], (3, 32, 32)))
  dataset = dataset.shuffle(100).batch(64)
  return dataset

def load_meta(filename):
  with open(filename, 'rb') as fo:
    metadata = load(fo, encoding='bytes')
  meta_mapping = dict()
  meta_mapping['coarse_labels'] = {i:metadata[b'coarse_label_names'][i].decode('utf-8')
                                  for i in range(len(metadata[b'coarse_label_names']))}

  meta_mapping['fine_labels'] = {i:metadata[b'fine_label_names'][i].decode('utf-8')
                                  for i in range(len(metadata[b'fine_label_names']))}
  return meta_mapping

In [109]:
train_dataset = extract_data(train_data_file)
train_dataset = transform_data(train_dataset)

test_dataset = extract_data(test_data_file)
test_dataset = transform_data(test_dataset)

metadata = load_meta(meta_data_file)

In [123]:
train_dataset.map(lambda x : x['images'])

TypeError: ignored

In [127]:
tf.reshape(train_dataset, (3, 32, 32))

ValueError: ignored